In [ ]:
import schedule
from datetime import datetime, timedelta, time
import time
import pandas as pd
import numpy as np
def compute_avg_price():
    try:
        global avg_price
        global price_same_time
        global now_string
        def make_identifier(df):
            str_id = df.apply(lambda x: '_'.join(map(str, x)), axis = 1)
            return pd.factorize(str_id)[0]

        df_merged = pd.read_csv('df_merged.csv')
        df_merged['date_time'] = pd.to_datetime(df_merged['scrape_datetime'], format="%d-%m-%Y %H:%M")
        df_merged['duration_outband'] = pd.to_timedelta(df_merged.out_duration)
        df_merged['duration_return'] = pd.to_timedelta(df_merged.ret_duration)
        df_merged.drop(['out_duration', 'ret_duration', 'scrape_datetime'], axis=1, inplace=True)
        df_merged['id'] = make_identifier(df_merged[['Company', 'outband_time', 'return_time', 'stops', 'duration_outband', 'duration_return', 'air_out', 'air_ret']])
        df_merged = df_merged.set_index('id')
        df_merged['scraped_at'] = df_merged['date_time'].astype('string')
        df_merged['scraped_at'] = [int(x[11:13]) for x in df_merged.scraped_at]

        avg_price = np.mean(df_merged['Price'])

        now = datetime.now()
        now_string = int(now.strftime("%H"))

        avg_hour_price = df_merged.groupby('scraped_at')['Price'].agg(np.mean).reset_index()
        price_same_time = int(avg_hour_price[avg_hour_price.scraped_at == now_string]['Price'].item())
    except:
        print('I will create a new data frame')
    return 'done'


def search_flight(departure='MIL', destination='NYC', date_dep='2022-11-28', date_ret='2022-12-02'):
    global flight_rows
    global companies
    global dep
    global duration
    global flight_hours
    global df
    global new_avg_price
    global advice_list
    
    # libraries
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from bs4 import BeautifulSoup
    import requests
    import time
    import os
    import re

    PATH = "/Users/manueldona/chromedriver"
    driver = webdriver.Chrome(PATH)

    link = 'https://www.kayak.it/flights/{departure}-{destination}/{date_dep}/{date_ret}?sort=bestflight_a'.format(departure=departure,destination=destination, date_dep=date_dep, date_ret=date_ret)
    driver.get(link)

    time.sleep(10)
    
    try:
        
        element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@class = "dDYU-close dDYU-mod-variant-default dDYU-mod-size-default"]'))
        )
        time.sleep(2)
        element.click() 
        
        time.sleep(20)

        flight_rows = WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "multi-row", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "option-text", " " ))]'))
        )


        companies = WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "codeshares-airline-names", " " ))]'))
        )

        dep = WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "times", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "top", " " ))]'))
        )


        duration = WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "stops", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "top", " " ))]'))
        )


        flight_hours = WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "allow-multi-modal-icons", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "top", " " ))]'))
        )


        airport = WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "times", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "bottom", " " ))]'))
        )

        advice = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '//*[contains(@id, "advice") and contains(@class, "value") and contains(@aria-busy, "false")]'))
        )
        

    except:
        driver.quit()
        
    
    time.sleep(7)

    # flight prices 
    try:

        prices_list = []

        for WebElement in flight_rows:
            elementHTML = WebElement.get_attribute('outerHTML')
            elementSoup = BeautifulSoup(elementHTML, 'html.parser')
            p = elementSoup.find("span", {"class": "price-text"})
            prices_list.append(p.text)

        prices = re.findall(r'\d{1,3}', str(prices_list))
        prices = [int(price) for price in prices]


        time.sleep(7)

        # company name

        companies_list = []

        for WebElement in companies:
            elementHTML = WebElement.get_attribute('outerHTML')
            elementSoup = BeautifulSoup(elementHTML, 'html.parser')
            c = elementSoup.find("span", {"class": "codeshares-airline-names"})
            companies_list.append(c.text)

        time.sleep(2)


        # flight time schedules
        dep_time = []

        for WebElement in dep:
            elementHTML = WebElement.get_attribute('outerHTML')
            elementSoup = BeautifulSoup(elementHTML, 'html.parser')
            d = elementSoup.find("div", {"class": "top"})
            dep_time.append(d.text)

        times = re.findall('\d{1,2}:\d\d', str(dep_time))

        times = [times[i:i+2] for i in range(0,len(times),2)]

        out = times[0::2]
        out = ['-'.join(x) for x in out]

        ret = times[1::2]
        ret = ['-'.join(x) for x in ret]

        # flight info's
        info = []

        for WebElement in duration:
            elementHTML = WebElement.get_attribute('outerHTML')
            elementSoup = BeautifulSoup(elementHTML, 'html.parser')
            i = elementSoup.find("span", {"class": "stops-text"})
            info.append(i.text.strip())

        time.sleep(4)

        # flight hours
        hours = []

        for WebElement in flight_hours:
            elementHTML = WebElement.get_attribute('outerHTML')
            elementSoup = BeautifulSoup(elementHTML, 'html.parser')
            h = elementSoup.find("div", {"class": "top"})
            hours.append(h.text.strip())

        out_dur = hours[0::2]
        ret_dur = hours[1::2]

    
        # airport name

        airports = []

        for WebElement in airport:
            elementHTML = WebElement.get_attribute('outerHTML')
            elementSoup = BeautifulSoup(elementHTML, 'html.parser')
            a = elementSoup.find("div", {"class": "bottom"})
            airports.append(a.text.strip())

        air = re.findall('[A-Z]{3}', str(airports))

        air_out_ret = [air[i:i+2] for i in range(0,len(air),2)]

        air_out = air_out_ret[0::2]
        air_out = ['-'.join(x) for x in air_out]

        air_ret = air_out_ret[1::2]
        air_ret = ['-'.join(x) for x in air_ret]
        
        # advice 
        advice_list = ""

        elementHTML = advice.get_attribute('outerHTML')
        elementSoup = BeautifulSoup(elementHTML, 'html.parser')
        ad = elementSoup.find("div", {"class": "value"})
        advice_list = ad.text.strip()

    except:
        pass



    # df creation
    import pandas as pd
    df = pd.DataFrame(list(zip(companies_list, prices, out, ret, info, out_dur, ret_dur, air_out, air_ret)),
                       columns =['Company', 'Price', 'outband_time', 'return_time', 'stops', 'out_duration', 'ret_duration', 'air_out', 'air_ret'])

    def make_identifier(df):
        str_id = df.apply(lambda x: '_'.join(map(str, x)), axis = 1)
        return pd.factorize(str_id)[0]

    df['combined_id'] = make_identifier(df[['Company', 'outband_time', 'return_time', 'stops', 'out_duration', 'ret_duration', 'air_out', 'air_ret']])
    
    df = df.sort_values('Price').drop_duplicates(subset=['combined_id'])
    
    df = df.iloc[:, :-1]
    
    from datetime import datetime

    now = datetime.now()

    # dd/mm/YY H:M:S
    dt_string_save = now.strftime("%d-%m-%Y_%H-%M")
    dt_string_df = now.strftime("%d-%m-%Y %H:%M")
    
    df['scrape_datetime'] = dt_string_df # adding scrape datetime to df

    df.to_csv('flights_{departure}_{destination}_{now}.csv'.format(departure=departure, destination=destination, now=dt_string_save), index=False)
    
    # saving avg price of df
    new_avg_price = np.mean(df['Price'])
    
    # adding new data to df_merged
    
    driver.quit()
    
    return(df)



def add_newdata():
    import pandas as pd
    from datetime import datetime
    import os

    global df_merged 
    
    now = datetime.now()
    dt_string_save = now.strftime("%d-%m-%Y_%H-%M")
    
    if os.path.exists('/Users/manueldona/df_merged.csv'):  # path for existing df_merged
        flights = pd.read_csv('df_merged.csv')
        print('I am adding df to the already existig data_merged.csv file')
    else:
        flights = pd.DataFrame() # creating a df_merged if it doesn't exist
        print('I am creating a new file called data_merged.csv')
    
    time.sleep(3) 
    
    df_merged = pd.concat([flights, df], axis=0, ignore_index=True) # adding new scraped data to df_merged
    df_merged.to_csv('df_merged.csv'.format(dt_string=dt_string_save), index=False) # saving changes to df_merged
    return(df_merged)
        

def sendmail():
    
    import smtplib

    # Import the email modules we'll need
    from email.message import EmailMessage

    import ssl
    
    try:
        var = (new_avg_price - avg_price)/avg_price

        var_same_time = (new_avg_price - price_same_time)/price_same_time

        if new_avg_price >= avg_price:
            string = 'At this time the mean price of {:.2f} is {:.2%} higher than usual.'.format(new_avg_price, var)
        else:
            string = 'At this time the mean price of {:.2f} is {:.2%} lower than usual.'.format(new_avg_price, var)

        if int(new_avg_price) >= int(price_same_time):
            string_2 = 'Usually at {} the price is {:.2%} lower'.format(now_string, var_same_time)
        else:
            string_2 = 'Usually at {} the price is {:.2%} lower'.format(now_string, var_same_time)
            
        body = """{string} {string_2}, the advice of kayak is: {}""".format(string=string, string_2=string_2, advice_list)
    
    except:
        body = """ I created a new data frame, now the average price is {:.2f}""".format(new_avg_price)

    email_sender = 'manu.md28@gmail.com'
    email_password = 'yegtifflhrkcjmzl'
    email_receiver = 'm.dona1@studenti.unibg.it'

    subject = 'Flight prices update'

    em = EmailMessage()
    em['From'] = email_sender
    em['To'] = email_receiver
    em['Subject'] = subject
    em.set_content(body)

    context = ssl.create_default_context()

    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(email_sender, email_password)
        smtp.sendmail(email_sender, email_receiver, em.as_string())
    return 'e-mail sent!'

In [ ]:
# scheduling the search_flight and add_newdata functions
from datetime import datetime
def job():
    compute_avg_price()
    time.sleep(1)
    search_flight()
    time.sleep(2)
    sendmail()
    time.sleep(1)
    add_newdata()
    print('Email sent')

schedule.every(3).hours.at(":30").do(job)
while True:
    schedule.run_pending()
    time.sleep(3)
    now = datetime.now()
    now_string = now.strftime("%d-%m-%Y %H:%M")
    if now_string == "31-10-2022 23:45":
        break
